### Data Preparation

In [1]:
import json,pandas as pd, numpy as np, os,csv,random,time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import csv,time
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from time import time

In [2]:
csv_file = '/home/zxt/data/all/dict_final.csv'
csv_data = pd.read_csv(csv_file, low_memory=False)
df = pd.DataFrame(csv_data)

In [3]:
len(df)

243760

In [4]:
df.head()

,image_id,label
0,S7_F07_R1_IMAG0140.JPG,elephant
1,CA-44_09_11_2015_CA-44_0007006.jpg,wild_boar
2,S10_F10_R1_IMAG1959.JPG,gazellethomosons
3,CA-41_08_04_2015_CA-41_0006076.jpg,squirrel
4,1603150213300952S194.JPG,hedgehog


In [5]:
# remove the damaged images
df = df[df['image_id']!='S7_T12_R1_IMAG0179.JPG']
df = df[df['image_id']!='S10_R09_R2_IMAG2759.JPG']

In [6]:
len(df)

243759

In [7]:
# Prepare all the animal class
elephant = df[df['label']=='elephant']
wild_boar = df[df['label']=='wild_boar']
gazellethomosons = df[df['label']=='gazellethomosons']
squirrel = df[df['label']=='squirrel']
hedgehog= df[df['label']=='hedgehog']

guineafowl = df[df['label']=='guineafowl']
moose = df[df['label']=='moose']
bird = df[df['label']=='bird']
fox = df[df['label']=='fox']

giraffe = df[df['label']=='giraffe']
buffalo = df[df['label']=='buffalo']
hare = df[df['label']=='hare']
vehicle = df[df['label']=='vehicle']
wildebeest = df[df['label']=='wildebeest']

cattle = df[df['label']=='cattle']
skunk = df[df['label']=='skunk']
lion = df[df['label']=='lion']
zebra = df[df['label']=='zebra']
black_bear = df[df['label']=='black_bear']

racoon = df[df['label']=='racoon']
coyote = df[df['label']=='coyote']
empty= df[df['label']=='empty']

In [27]:
# ramdom sampling
elephant_2k = elephant.sample(n=500,random_state=912,replace=False)
wild_boar_2k = wild_boar.sample(n=500,random_state=912,replace=False)
gazellethomosons_2k = gazellethomosons.sample(n=500,random_state=912,replace=False)
squirrel_2k = squirrel.sample(n=500,random_state=912,replace=False)
hedgehog_2k = hedgehog.sample(n=500,random_state=912,replace=False)

guineafowl_2k = guineafowl.sample(n=500,random_state=912,replace=False)
moose_2k = moose.sample(n=500,random_state=912,replace=False)
bird_2k = bird.sample(n=500,random_state=912,replace=False)
fox_2k = fox.sample(n=500,random_state=912,replace=False)

giraffe_2k = giraffe.sample(n=500,random_state=912,replace=False)
buffalo_2k = buffalo.sample(n=500,random_state=912,replace=False)
hare_2k = hare.sample(n=500, random_state=912,replace=False)
vehicle_2k = vehicle.sample(n=500,random_state=912,replace=False)
wildebeest_2k = wildebeest.sample(n=500,random_state=912,replace=False)

cattle_2k = cattle.sample(n=500,random_state=912,replace=False)
skunk_2k = skunk.sample(n=500,random_state=912,replace=False)
lion_2k = lion.sample(n=500,random_state=912,replace=False)
zebra_2k = zebra.sample(n=500,random_state=912,replace=False)
black_bear_2k = black_bear.sample(n=500,random_state=912,replace=False)

racoon_2k = racoon.sample(n=500,random_state=912,replace=False)
coyote_2k = coyote.sample(n=500,random_state=912,replace=False)
empty_2k = empty.sample(n=500,random_state=912,replace=False)

In [28]:
len(cattle_2k)

500

In [29]:
# randomly check the number of each class
cattle_2k.head(2)

,image_id,label
31147,CA-10_08_13_2015_CA-10_0011818.jpg,cattle
53841,CA-43_06_21_2016_CA-43_0022339.JPG,cattle


In [30]:
# generate a mining dataframe
df_all = pd.concat([empty_2k,racoon_2k,coyote_2k,elephant_2k,wild_boar_2k,gazellethomosons_2k,squirrel_2k,hedgehog_2k,guineafowl_2k,moose_2k,bird_2k,fox_2k,giraffe_2k,buffalo_2k,hare_2k,vehicle_2k,wildebeest_2k,cattle_2k,skunk_2k,lion_2k,zebra_2k,black_bear_2k ])

In [31]:
len(df_all)

11000

In [13]:
# dcheck if the dataset is balanced
df_all['label'].value_counts().plot.bar()

In [14]:
imglist =list(df_all['image_id'])

In [32]:
df_all.to_csv('/home/zxt/data/final_22/ml/df_all_500.csv')

In [64]:
# split train, validate, test datasets
train_df, validate0_df = train_test_split(df_all, test_size=0.20, random_state=522)
train_df = train_df.reset_index(drop=True)
validate0_df = validate0_df.reset_index(drop=True)

In [65]:
validate_df, test_df = train_test_split(validate0_df, test_size=0.50, random_state=626)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [66]:
len(train_df)

35200

### train set

In [67]:
# generate desired pixels dataset
imglist = train_df['image_id'].tolist()
X_train = np.empty((len(imglist),32,32,3))
X_train.shape

(35200, 32, 32, 3)

In [68]:
Y_train = np.empty((len(imglist),1))
Y_train.shape

(35200, 1)

In [69]:
start = time()

path = '/home/zxt/data/all/resized_data/'
count =0
nonelist=[]
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(32,32))
        X_train[count]=img
        count+=1
        if count % 3000 == 0:
            print(count)
    except:
        nonelist.append(img_path)
        
end = time()
print("Running time: %s seconds"%(end - start))

3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
Running time: 66.02647614479065 seconds


In [70]:
len(nonelist)

0

In [71]:
nonelist

[]

In [81]:
label = train_df['label']
Y_train = [i for i in label]
Y_train = np.array(Y_train)
Y_train = Y_train.reshape(len(Y_train),1)
Y_train.shape

(35200, 1)

In [82]:
Y_train

array([['hedgehog'],
       ['wild_boar'],
       ['hedgehog'],
       ...,
       ['coyote'],
       ['squirrel'],
       ['racoon']], dtype='<U16')

In [83]:
# X_train[0]

In [84]:
len(Y_train)

35200

In [85]:
len(X_train)

35200

In [87]:
np.save('/home/zxt/data/final_22/32/2k/X_train',X_train)
np.save('/home/zxt/data/final_22/32/2k/Y_train',Y_train)

### validate set

In [72]:
validate_df.shape

(4400, 2)

In [73]:
imglist = validate_df['image_id'].tolist()
X_validate = np.empty((len(imglist),32,32,3))
X_validate.shape

(4400, 32, 32, 3)

In [74]:
start = time()
count=0
nonelist=[]
path = '/home/zxt/data/all/resized_data/'
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(32,32))
        X_validate[count]=img
        count+=1
        if count % 3000 ==0:
            print(count)
    except:
        nonelist.append(img_path)
        
end = time()
print("Running time: %s seconds"%(end - start))

3000
Running time: 102.75164842605591 seconds


In [75]:
len(nonelist)

0

In [76]:
nonelist

[]

In [88]:
label = validate_df['label']
Y_validate = [i for i in label]
Y_validate = np.array(Y_validate)
Y_validate = Y_validate.reshape(len(Y_validate),1)
Y_validate.shape

(4400, 1)

In [89]:
X_validate[0]

array([[[254., 249., 253.],
        [254., 249., 255.],
        [253., 248., 254.],
        ...,
        [210., 226., 255.],
        [218., 225., 251.],
        [250., 250., 250.]],

       [[254., 249., 253.],
        [254., 250., 251.],
        [254., 250., 251.],
        ...,
        [255., 247., 248.],
        [250., 250., 255.],
        [252., 251., 255.]],

       [[243., 243., 253.],
        [254., 249., 253.],
        [254., 249., 253.],
        ...,
        [255., 249., 253.],
        [249., 250., 255.],
        [237., 240., 247.]],

       ...,

       [[ 98., 106.,  65.],
        [ 59.,  65.,  39.],
        [ 33.,  53.,   2.],
        ...,
        [ 76., 113.,  44.],
        [ 51.,  91.,  28.],
        [ 53.,  82.,  28.]],

       [[ 84.,  90.,  62.],
        [ 71.,  82.,  52.],
        [ 64.,  75.,  45.],
        ...,
        [ 55.,  74.,  44.],
        [ 48.,  66.,  40.],
        [ 72.,  97.,  39.]],

       [[164., 164., 166.],
        [210., 208., 211.],
        [176., 1

In [90]:
Y_validate

array([['zebra'],
       ['bird'],
       ['empty'],
       ...,
       ['moose'],
       ['skunk'],
       ['moose']], dtype='<U16')

In [91]:
np.save('/home/zxt/data/final_22/32/2k/X_validate',X_validate)
np.save('/home/zxt/data/final_22/32/2k/Y_validate',Y_validate)

### test set

In [77]:
imglist = test_df['image_id'].tolist()
X_test = np.empty((len(imglist),32,32,3))
X_test.shape

(4400, 32, 32, 3)

In [78]:
start=time()

count=0
nonelist=[]
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(32,32))
        X_test[count]=img
        count+=1
        if count % 3000 == 0:
            print(count)
    except:
        nonelist.append(img_path)
        
end=time()
print("Runing time: %s seconds"%(end-start))

3000
Runing time: 102.87826251983643 seconds


In [79]:
len(nonelist)

0

In [80]:
nonelist

[]

In [94]:
label = test_df['label']
Y_test = [i for i in label]
Y_test = np.array(Y_test)
Y_test = Y_test.reshape(len(Y_test),1)
Y_test.shape

(4400, 1)

In [95]:
test_df[:2]

,image_id,label
0,S7_C11_R2_IMAG1269.JPG,guineafowl
1,S7_M09_R1_IMAG0181.JPG,lion


In [96]:
Y_test[:2]

array([['guineafowl'],
       ['lion']], dtype='<U16')

In [97]:
X_test[0]

array([[[113., 131., 155.],
        [118., 134., 159.],
        [117., 133., 158.],
        ...,
        [172., 185., 191.],
        [155., 168., 177.],
        [156., 170., 181.]],

       [[124., 138., 164.],
        [115., 131., 156.],
        [119., 135., 160.],
        ...,
        [143., 160., 168.],
        [137., 153., 166.],
        [131., 149., 163.]],

       [[126., 139., 158.],
        [126., 138., 164.],
        [131., 139., 160.],
        ...,
        [132., 150., 164.],
        [123., 140., 156.],
        [109., 129., 153.]],

       ...,

       [[154., 109.,  67.],
        [133.,  90.,  47.],
        [170., 132.,  87.],
        ...,
        [107.,  76.,  32.],
        [103.,  62.,  16.],
        [ 92.,  50.,  10.]],

       [[137., 111.,  88.],
        [105.,  78.,  57.],
        [138., 112.,  87.],
        ...,
        [ 96.,  75.,  48.],
        [137., 116.,  85.],
        [ 83.,  66.,  46.]],

       [[206., 208., 207.],
        [185., 186., 190.],
        [214., 2

In [100]:
np.save('/home/zxt/data/final_22/32/2k/X_test',X_test)
np.save('/home/zxt/data/final_22/32/2k/Y_test',Y_test)